'''Selenium Web scrapping'''

In [1]:
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common import keys
import time
driver=webdriver.Chrome()
f=[]
h=['https://m.cricbuzz.com/cricket-series/7607/indian-premier-league-2024/matches','https://m.cricbuzz.com/cricket-series/5945/indian-premier-league-2023/matches','https://m.cricbuzz.com/cricket-series/4061/indian-premier-league-2022/matches',"https://m.cricbuzz.com/cricket-series/3472/indian-premier-league-2021/matches",'https://m.cricbuzz.com/cricket-series/3130/indian-premier-league-2020/matches','https://m.cricbuzz.com/cricket-series/2810/indian-premier-league-2019/matches','https://m.cricbuzz.com/cricket-series/2676/indian-premier-league-2018/matches','https://m.cricbuzz.com/cricket-series/2568/indian-premier-league-2017/matches','https://m.cricbuzz.com/cricket-series/2430/indian-premier-league-2016/matches','https://m.cricbuzz.com/cricket-series/2330/indian-premier-league-2015/matches','https://m.cricbuzz.com/cricket-series/2261/indian-premier-league-2014/matches','https://m.cricbuzz.com/cricket-series/2170/indian-premier-league-2013/matches','https://m.cricbuzz.com/cricket-series/2115/indian-premier-league-2012/matches','https://m.cricbuzz.com/cricket-series/2037/indian-premier-league-2011/matches','https://m.cricbuzz.com/cricket-series/2060/indian-premier-league-2010/matches','https://m.cricbuzz.com/cricket-series/2059/indian-premier-league-2009/matches','https://m.cricbuzz.com/cricket-series/2058/indian-premier-league-2008/matches']
for j in h:
    driver.get(j)
    time.sleep(10)
    try:
        a=driver.find_elements(By.CSS_SELECTOR,'div[class="mt-2"]')
    except:
        print('no found')
    try:        

        for i in a:

            date=i.find_element(By.CSS_SELECTOR,"div[class='mt-2']>a>div>span").text
            e= i.find_element(By.CSS_SELECTOR,"div[class='flex flex-col gap-3 my-2  ']>div:nth-child(1)>div>span:nth-child(2)").text
            b=i.find_element(By.CSS_SELECTOR,"div[class='flex flex-col gap-3 my-2  ']>div:nth-child(2)>div>span:nth-child(2)").text
            c=i.find_element(By.CSS_SELECTOR,"div[class='flex justify-between items-center']>span").text
            d=i.find_element(By.CLASS_NAME,'text-cbComplete').text
            h=i.find_element(By.CLASS_NAME,'font-medium').text
            m=i.find_element(By.CSS_SELECTOR,'div[class="flex items-center gap-4 justify-between"]:nth-child(2)>span').text
            f.append({
                'Date':date,
                'Team A':e,
                'Team B':b,
                'Stadium':c,
                "1st team score":h,
                '2nd team score':m,
                'margin':d

            })
            #driver.close()
    except:
        print('not found')  

'''push the data to mysql Database'''

In [ ]:
from sqlalchemy import create_engine
host="localhost"
database="cricket"
username="root"
port="3306"
password="seenu2218"

engine=f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
engine=create_engine(engine)
tablename='ipl1'
f.to_sql(tablename,engine,if_exists='replace',index=False)


In [ ]:
'''Taken the Data from Mysql DataBase'''

In [ ]:
import mysql.connector
import pandas as pd
conn=mysql.connector.connect(
    host='localhost',
    username='root',
    port='3306',
    password='seenu2218',
    database='Cricket'

)
cursor=conn.cursor()
cursor.execute('select * from ipl1')
b=cursor.fetchall()
cursor.execute('select column_name from information_schema.columns where table_name="ipl1" order by ordinal_position')
column=cursor.fetchall()
columna=[flat[0] for flat in column]
Data=pd.DataFrame(b,columns=columna)

In [ ]:
'''Cleaning the data''' 

In [ ]:
Data[['scoreA','wicketA']] = Data['1st team score'].str.split('-', expand=True).fillna('10')
Data[['scoreB','wicketB']] = Data['2nd team score'].str.split('-', expand=True).fillna('10')
Data['wicket A']=Data['wicketA'].str.replace(r'(\d*).*',r'\1',regex=True)
Data['wicket B']=Data['wicketB'].str.replace(r'(\d*).*',r'\1',regex=True)
Data['overs B'] = Data['wicketB'].str.replace(r'\d+\s*\((\d+\.\d+|\d+)\)', r'\1', regex=True)

Data['overs A'] = Data['wicketA'].str.replace(r'\d+\s*\((\d+\.\d+|\d+)\)', r'\1', regex=True)
Data = Data.drop(columns=['wicketA', 'wicketB'])
Data['wicket A']=Data['wicket A'].astype('int64')
Data['wicket B']=Data['wicket B'].astype('int64')
Data['overs A'] =Data['overs A'] .astype('float64')
Data['overs B'] =Data['overs B'] .astype('float64')


In [ ]:
x=Data[Data['wicket A']==10]
y=Data[Data['wicket B']==10]
x[['scoreA','overs A']]=x['1st team score'].str.extract(r'(\d+)\s*\((\d+\.\d+|\d+)')
y[['scoreB','overs B']]=y['2nd team score'].str.extract(r'(\d+)\s*\((\d+\.\d+|\d+)')
Data=Data[(Data['wicket A']!=10)]
Data=Data[(Data['wicket B']!=10)]
Data1=pd.concat([Data,x,y])
Data1=Data1.drop_duplicates()

In [ ]:
Data1=Data1.reset_index()
Data1 = Data1.drop_duplicates(subset='s.no')
Data1=Data1.sort_values(by='s.no')
Data1=Data1.set_index('s.no')
z=Data1[Data1['overs B']==10]
z[['scoreB','overs B']]=z['2nd team score'].str.extract(r'(\d+)\s*\((\d+\.\d+|\d+)')
Data1=Data1[Data1['overs B']!=10]

In [ ]:
Data2=pd.concat([Data1,z])
Data2=Data2.reset_index()
Data2=Data2.sort_values(by='s.no')
Data2=Data2.set_index('s.no')

In [ ]:
Data2['scoreA'] = Data2['scoreA'].fillna(0).astype('int64')
Data2['scoreB'] = Data2['scoreB'].fillna(0).astype('int64')
Data2['wicket A'] = Data2['wicket A'].fillna(0).astype('int64')
Data2['wicket B'] = Data2['wicket B'].fillna(0).astype('int64')
Data2['overs A'] =Data2['overs A'].fillna(0).astype('float64')
Data2['overs B'] =Data2['overs B'].fillna(0).astype('float64')

In [ ]:
Data2['scoreA'] = Data2['scoreA'].astype(str)
Data2['scoreB'] = Data2['scoreB'].astype(str)
Data2['scoreA']=Data2['scoreA'].str.strip()
Data2['scoreB']=Data2['scoreB'].str.strip()
Data2['scoreA'] = Data2['scoreA'].astype('int64')
Data2['scoreB'] = Data2['scoreB'].astype('int64')
Data2['Team1'] = Data2.apply(lambda row: 'won' if row['scoreA'] > row['scoreB'] else 'lost', axis=1)
Data2['Team2'] = Data2.apply(lambda row: 'won' if row['scoreA'] < row['scoreB'] else 'lost', axis=1)

In [ ]:
'''resent the data'''

In [ ]:
engine=f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
engine=create_engine(engine)
tablename='ipl2'
Data2.to_sql(tablename,engine,if_exists='replace',index=False)